# Prepare Environment

In [1]:
import os
from pprint import pprint
import io
from datetime import timedelta
from importlib import reload

import adalib

from pylab import rcParams
import matplotlib.pyplot as plt

%matplotlib inline
import numpy as np
import seaborn as sns
import statsmodels.api as sm
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt
import statsmodels.api as sm
from statsmodels.tsa.ar_model import AR
from sklearn.model_selection import TimeSeriesSplit
from sklearn import metrics
import pandas as pd
import math
from statsmodels.tsa.api import Holt
from fbprophet import Prophet
from fbprophet.diagnostics import cross_validation, performance_metrics
from fbprophet.plot import (
    plot_cross_validation_metric,
    plot_forecast_component,
)

# Acquire

In [2]:
df = pd.read_csv('precipitation.csv')

# Summarize Data

In [3]:
adalib.summarize(df)

RANDOM SAMPLE OF 10
        Unnamed: 0      STATION        DATE  PRCP
1018          1018  US1TXCML008  2018-03-29  1.01
130012      130012  USC00410902  1977-10-09  0.03
107016      107016  USC00417628  2004-12-31  0.00
162841      162841  USC00416276  1957-07-10  0.00
15920        15920  US1TXAT0026  2018-04-21  0.03
217413      217413  USC00416276  1910-12-17  0.05
142555      142555  USC00410902  1968-06-15  0.00
147405      147405  USC00410902  1964-03-23  0.00
163070      163070  USC00416276  1958-02-24  0.00
18678        18678  US1TXCML141  2018-10-21  0.01

SHAPE: (232574, 4)

DESCRIPTION
          Unnamed: 0           PRCP
count  232574.000000  229661.000000
mean   116286.500000       0.099676
std     67138.475091       0.393292
min         0.000000       0.000000
25%     58143.250000       0.000000
50%    116286.500000       0.000000
75%    174429.750000       0.000000
max    232573.000000      11.500000

INFORMATION
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 232574 ent

In [4]:
adalib.df_missing_vals_by_col(df)

,nmissing,percentage,nempty
Unnamed: 0,0,0.000000,0
STATION,0,0.000000,0
DATE,0,0.000000,0
PRCP,2913,1.252505,0


In [35]:
df.set_index('DATE')

station_counts = df.groupby('STATION').count()
station_counts = station_counts.sort_values(by=['PRCP'], ascending=True)
print(station_counts.head())
print(station_counts.tail(10))

             Unnamed: 0  DATE  PRCP
STATION                            
US1TXBXR362         101   101   101
US1TXGP0126         106   106   106
US1TXGP0103         107   107   107
US1TXBXR073         108   108   108
US1TXCML038         127   127   127
             Unnamed: 0   DATE   PRCP
STATION                              
US1TXBXR027        4402   4402   4394
US1TXBXR015        4407   4407   4407
US1TXAT0001        4487   4487   4487
US1TXCML008        4600   4600   4576
USW00012909        8346   8346   8337
USC00411777        8962   8962   8949
USC00415454       10562  10562  10551
USC00416276       29087  29087  28003
USC00417628       33026  33026  32855
USC00410902       44512  44512  44475


USW00012909 is the longest running station for collecting precipitation data in San Antonio. I will use this station alone for my initial model.

In [38]:
df_initial = df[(df['STATION'] == 'USW00012909')]

In [40]:
print(df_initial.DATE.max())
print(df_initial.DATE.min())

2019-05-15
1949-01-01


In [45]:
df_initial.dtypes

Unnamed: 0      int64
STATION        object
DATE           object
PRCP          float64
dtype: object

In [48]:
df_initial['DATE'] = pd.to_datetime(df_initial['DATE'])

/usr/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [49]:
df_initial.dtypes

Unnamed: 0             int64
STATION               object
DATE          datetime64[ns]
PRCP                 float64
dtype: object

Dropping Nans from the dataframe to feed into initial model. 

In [55]:
df_initial.dropna(inplace=True)
df_initial.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8337 entries, 5370 to 178607
Data columns (total 4 columns):
Unnamed: 0    8337 non-null int64
STATION       8337 non-null object
DATE          8337 non-null datetime64[ns]
PRCP          8337 non-null float64
dtypes: datetime64[ns](1), float64(1), int64(1), object(1)
memory usage: 325.7+ KB


/usr/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


## Train/Test Split

In [51]:
aggregation = "sum"

train = df_initial[:"2005-01"].resample("M").agg(aggregation)
test = df_initial["2005-02":].resample("M").agg(aggregation)

TypeError: cannot do slice indexing on <class 'pandas.core.indexes.numeric.Int64Index'> with these indexers [2005-01] of <class 'str'>

In [ ]:
print("Observations: %d" % (len(train.values) + len(test.values)))
print("Training Observations: %d" % (len(train)))
print("Testing Observations: %d" % (len(test)))

## Modeling

In [36]:
def plot_data_and_predictions(train, test, predictions, label):
    plt.figure(figsize=(10, 8))

    plt.plot(train,label='Train')
    plt.plot(test, label='Test')
    plt.plot(predictions, label=label, linewidth=5)

    plt.legend(loc='best')
    plt.show()


def evaluate(actual, predictions, output=True):
    mse = metrics.mean_squared_error(actual, predictions)
    rmse = math.sqrt(mse)

    if output:
        print('MSE:  {}'.format(mse))
        print('RMSE: {}'.format(rmse))
    else:
        return mse, rmse    

def plot_and_eval(train, test, predictions, actual, metric_fmt='{:.2f}', linewidth=4):
    if type(predictions) is not list:
        predictions = [predictions]

    plt.figure(figsize=(16, 8))
    plt.plot(train, label='Train')
    plt.plot(test, label='Test')

    for yhat in predictions:
        mse, rmse = evaluate(actual, yhat, output=False)        
        label = f'{yhat.name}'
        if len(predictions) > 1:
            label = f'{label} -- MSE: {metric_fmt} RMSE: {metric_fmt}'.format(mse, rmse)
        plt.plot(yhat, label=label, linewidth=linewidth)
        plt.title(f"{train.name}")

    if len(predictions) == 1:
        label = f'{label} -- MSE: {metric_fmt} RMSE: {metric_fmt}'.format(mse, rmse)
        plt.title(f"{train.name}\n{label}")

    plt.legend(loc='best')
    plt.show()